In [28]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV, train_test_split

import numpy as np
import pandas as pd

In [29]:

# 데이터와 타겟(종속 변수) 준비
X = pd.read_csv('../X_Features_Final.csv')# 독립 변수 데이터
y = pd.read_csv('../y_open_0905~_1000(5min)/y_open_0910.csv')# 종속 변수 데이터

# 시간 형태로 변경
X['date'] = pd.to_datetime(X['date'] ,format="%Y-%m-%d")
y['date'] = pd.to_datetime(y['date'],format= "%Y%m%d")

df = pd.merge(X,y, on='date',how='inner')
df.sort_index(ascending=False,inplace=True)
df.drop(columns='Unnamed: 0',inplace=True)
df.set_index('date',inplace=True)


In [30]:
# X와 y 정의
X = df.drop('open', axis=1)
y = df['open']

# 분리 전 백테스트를 위한 date 생성
dates = df.index

# 학습셋 분리 (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False, random_state=42)

In [36]:
# 선형 회귀 모델 초기화
model = LinearRegression()

# 탐색할 하이퍼파라미터 그리드 정의
param_grid = {
    'fit_intercept': [True, False],  # fit_intercept 파라미터를 탐색
}

# GridSearchCV를 사용하여 최적 파라미터 탐색
grid_search = GridSearchCV(model, param_grid, scoring='neg_mean_squared_error', cv=5)
grid_search.fit(X_train, y_train)

# 최적 파라미터 출력
print("최적 파라미터:", grid_search.best_params_)

# 최적 모델 선택
best_model_LR = grid_search.best_estimator_

# 예측을 수행
y_pred = best_model_LR.predict(X_train)

# RMSE 계산
mse = mean_squared_error(y_train, y_pred)
r2 = r2_score(y_train, y_pred)
rmse = np.sqrt(mean_squared_error(y_train, y_pred))
print("MSE:", mse ,"RMSE:", rmse, "r2_score:",r2)


최적 파라미터: {'fit_intercept': True}
MSE: 135900.04230540668 RMSE: 368.64622920275025 r2_score: 0.9964491836420264


In [37]:
# 검증
y_pred = best_model_LR.predict(X_test)

# RMSE 계산
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("MSE:", mse ,"RMSE:", rmse, "r2_score:",r2)

MSE: 89016.91035103767 RMSE: 298.3570182701216 r2_score: 0.9639460887221531


#DecisionTree

In [43]:
from sklearn.tree import DecisionTreeRegressor

# 의사 결정 트리 모델 초기화
model = DecisionTreeRegressor()

# 탐색할 하이퍼파라미터 그리드 정의
param_grid = {
    'max_depth': [None, 10, 20, 30],  # 트리의 최대 깊이
    'min_samples_split': [2, 5, 10],  # 분할을 위한 최소 샘플 수
    'min_samples_leaf': [1, 2, 4]  # 리프 노드에 필요한 최소 샘플 수
}

# GridSearchCV를 사용하여 최적 파라미터 탐색 (평균 제곱 오차를 최소화)
grid_search = GridSearchCV(model, param_grid, scoring='neg_mean_squared_error', cv=5)
grid_search.fit(X_train, y_train)

# 최적 파라미터 출력
print("최적 파라미터:", grid_search.best_params_)

# 최적 모델 선택
best_model_DT = grid_search.best_estimator_

# 최적 모델을 사용하여 테스트 데이터 세트에 대한 예측을 수행
y_pred = best_model_DT.predict(X_test)

# RMSE 계산
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("MSE:", mse ,"RMSE:", rmse, "r2_score:",r2)

최적 파라미터: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2}
MSE: 364990.0958333333 RMSE: 604.1441018774688 r2_score: 0.8521705541051232


In [44]:
from sklearn.ensemble import RandomForestRegressor

# 랜덤 포레스트 모델 초기화
model = RandomForestRegressor()

# 탐색할 하이퍼파라미터 그리드 정의
param_grid = {
    'n_estimators': [50, 100, 200],  # 트리의 개수
    'max_depth': [None, 10, 20, 30],  # 트리의 최대 깊이
    'min_samples_split': [2, 5, 10],  # 분할을 위한 최소 샘플 수
    'min_samples_leaf': [1, 2, 4]  # 리프 노드에 필요한 최소 샘플 수
}

# GridSearchCV를 사용하여 최적 파라미터 탐색 (평균 제곱 오차를 최소화)
grid_search = GridSearchCV(model, param_grid, scoring='neg_mean_squared_error', cv=5)
grid_search.fit(X_train, y_train)

# 최적 파라미터 출력
print("최적 파라미터:", grid_search.best_params_)

# 최적 모델 선택
best_model_RF = grid_search.best_estimator_

# 최적 모델을 사용하여 테스트 데이터 세트에 대한 예측을 수행
y_pred = best_model_RF.predict(X_test)

# RMSE 계산
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("MSE:", mse ,"RMSE:", rmse, "r2_score:",r2)

In [ ]:
import xgboost as xgb

# XGBoost 모델 초기화
model = xgb.XGBRegressor(objective='reg:squarederror')  # 회귀 문제 설정

# 탐색할 하이퍼파라미터 그리드 정의
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 7, 9],
    'learning_rate': [0.01, 0.05, 0.1],
    'subsample': [0.8, 0.9, 1.0],
    'colsample_bytree': [0.8, 0.9, 1.0]
}

# GridSearchCV를 사용하여 최적 파라미터 탐색 (평균 제곱 오차를 최소화)
grid_search = GridSearchCV(model, param_grid, scoring='neg_mean_squared_error', cv=5, verbose=1)
grid_search.fit(X_train, y_train)

# 최적 파라미터 출력
print("최적 파라미터:", grid_search.best_params_)

# 최적 모델 선택
best_model_XGB = grid_search.best_estimator_

# 최적 모델을 사용하여 테스트 데이터 세트에 대한 예측을 수행
y_pred = best_model_XGB.predict(X_test)

# RMSE 계산
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("MSE:", mse ,"RMSE:", rmse, "r2_score:",r2)


In [ ]:
import lightgbm as lgb

# LightGBM 모델 초기화
model = lgb.LGBMRegressor()

# 탐색할 하이퍼파라미터 그리드 정의
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [5, 10, 15],
    'learning_rate': [0.01, 0.05, 0.1],
    'subsample': [0.8, 0.9, 1.0],
    'colsample_bytree': [0.8, 0.9, 1.0]
}

# GridSearchCV를 사용하여 최적 파라미터 탐색 (평균 제곱 오차를 최소화)
grid_search = GridSearchCV(model, param_grid, scoring='neg_mean_squared_error', cv=5)

grid_search.fit(X_train, y_train)

# 최적 파라미터 출력
print("최적 파라미터:", grid_search.best_params_)

# 최적 모델 선택
best_model_LGB = grid_search.best_estimator_

# 최적 모델을 사용하여 테스트 데이터 세트에 대한 예측을 수행
y_pred = best_model_LGB.predict(X_test)

# RMSE 계산
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("MSE:", mse ,"RMSE:", rmse, "r2_score:",r2)

In [ ]:
from sklearn.svm import SVR

# SVM 모델 초기화
model = SVR()

# 탐색할 하이퍼파라미터 그리드 정의
param_grid = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf'],
    'gamma': ['scale', 'auto']
}

# GridSearchCV를 사용하여 최적 파라미터 탐색 (평균 제곱 오차를 최소화)
grid_search = GridSearchCV(model, param_grid, scoring='neg_mean_squared_error', cv=5)
grid_search.fit(X_train, y_train)

# 최적 파라미터 출력
print("최적 파라미터:", grid_search.best_params_)

# 최적 모델 선택
best_model_SVR = grid_search.best_estimator_

# 최적 모델을 사용하여 테스트 데이터 세트에 대한 예측을 수행
y_pred = best_model_SVR.predict(X_test)

# RMSE 계산
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("MSE:", mse ,"RMSE:", rmse, "r2_score:",r2)